In [22]:
import os
from sklearn.model_selection import train_test_split

# Directory containing text files
data_dir = 'data'
all_text = ''

# Reading all text files from the directory
for file_name in os.listdir(data_dir):
    if file_name.endswith('.txt'):
        with open(os.path.join(data_dir, file_name), 'r', encoding='utf-8') as file:
            all_text += file.read() + '\n'

# Splitting data into training (80%) and evaluation (20%) sets
train_text, eval_text = train_test_split(all_text.split('\n'), test_size=0.2, random_state=42)

# Joining lines to form complete text for training and evaluation
train_text = '\n'.join(train_text)
eval_text = '\n'.join(eval_text)

# Saving the training and evaluation data to separate files
with open('train.txt', 'w', encoding='utf-8') as train_file:
    train_file.write(train_text)

with open('eval.txt', 'w', encoding='utf-8') as eval_file:
    eval_file.write(eval_text)

print("Data split and saved successfully.")


Data split and saved successfully.


In [6]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# Load tokenizer and model (you can replace 'gpt2' with any other LLM)
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Load the text files into datasets
train_dataset = load_dataset('text', data_files='train.txt')['train']
eval_dataset = load_dataset('text', data_files='eval.txt')['train']

from transformers import DataCollatorForLanguageModeling

# Set pad_token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the datasets and include labels
def tokenize_function(examples):
    outputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)
    outputs["labels"] = outputs["input_ids"].copy()  # Set labels to be the same as input_ids for causal LM
    return outputs

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# Define a data collator that will dynamically pad the inputs and labels
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to False for causal language modeling
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')

print("Fine-tuning complete and model saved.")



Map: 100%|██████████| 15/15 [00:00<00:00, 833.27 examples/s]
C:\Users\Filip\AppData\Local\Temp\ipykernel_17420\239879204.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


                                      
                                            

  0%|          | 0/42 [10:15<?, ?it/s]       



{'eval_loss': 4.82485294342041, 'eval_runtime': 0.3054, 'eval_samples_per_second': 49.118, 'eval_steps_per_second': 13.098, 'epoch': 1.0}




                                      
                                            

  0%|          | 0/42 [10:19<?, ?it/s]       



{'eval_loss': 4.740758895874023, 'eval_runtime': 0.3216, 'eval_samples_per_second': 46.636, 'eval_steps_per_second': 12.436, 'epoch': 2.0}





                                      
                                            

  0%|          | 0/42 [10:25<?, ?it/s]       

                                      
100%|██████████| 42/42 [00:14<00:00,  2.91it/s]


{'eval_loss': 4.708907604217529, 'eval_runtime': 0.402, 'eval_samples_per_second': 37.312, 'eval_steps_per_second': 9.95, 'epoch': 3.0}
{'train_runtime': 14.4416, 'train_samples_per_second': 11.633, 'train_steps_per_second': 2.908, 'train_loss': 4.7641652425130205, 'epoch': 3.0}
Fine-tuning complete and model saved.


In [7]:
from transformers import Trainer, TrainingArguments, AutoModelForMaskedLM, AutoTokenizer
from datasets import load_dataset

# Load tokenizer and model (using 'xlm-roberta-base' for multilingual capabilities)
model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Load the text files into datasets
train_dataset = load_dataset('text', data_files='train.txt')['train']
eval_dataset = load_dataset('text', data_files='eval.txt')['train']

from transformers import DataCollatorForLanguageModeling

# Set pad_token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the datasets and include labels
def tokenize_function(examples):
    outputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)
    outputs["labels"] = outputs["input_ids"].copy()  # Set labels to be the same as input_ids for masked LM
    return outputs

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# Define a data collator that will dynamically pad the inputs and labels
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,  # Enable masked language modeling for training
    mlm_probability=0.15  # Set the probability of masking tokens
)

# Define training arguments with constraints for limited VRAM
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,  # Adjust batch size as needed
    per_device_eval_batch_size=2,   # Adjust as needed
    gradient_accumulation_steps=4,  # Increase to simulate a larger batch size with less VRAM
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=500,
    fp16=True,  # Use mixed precision for better VRAM efficiency
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')

print("Fine-tuning complete and model saved.")


c:\Users\Filip\anaconda3\envs\ASP\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Filip\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['rober

{'eval_loss': 2.205465078353882, 'eval_runtime': 1.0121, 'eval_samples_per_second': 14.82, 'eval_steps_per_second': 7.904, 'epoch': 1.0}







                                      
                                            

  0%|          | 0/42 [26:37<?, ?it/s]       



{'eval_loss': 2.551729917526245, 'eval_runtime': 1.1791, 'eval_samples_per_second': 12.722, 'eval_steps_per_second': 6.785, 'epoch': 2.0}







                                      
                                            

  0%|          | 0/42 [27:06<?, ?it/s]       



{'eval_loss': 2.179887056350708, 'eval_runtime': 1.046, 'eval_samples_per_second': 14.34, 'eval_steps_per_second': 7.648, 'epoch': 3.0}


                                      
100%|██████████| 21/21 [01:17<00:00,  3.68s/it]


{'train_runtime': 77.2222, 'train_samples_per_second': 2.176, 'train_steps_per_second': 0.272, 'train_loss': 2.478679656982422, 'epoch': 3.0}
Fine-tuning complete and model saved.


In [12]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

# Step 1: Load the fine-tuned model and tokenizer
model_path = './fine-tuned-model'
model = AutoModelForMaskedLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # Set the model to evaluation mode

# Step 2: Function to run inference and predict masked tokens
def predict_masked_token(text):
    # Tokenize input with <mask>
    inputs = tokenizer(text, return_tensors="pt")
    
    # Generate predictions
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits
    
    # Find the index of the masked token and get the predicted token
    masked_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
    predicted_token_id = torch.argmax(predictions[0, masked_index, :])
    predicted_token = tokenizer.decode(predicted_token_id)
    
    return predicted_token

# Step 3: Use the function to answer a question or complete a sentence
example_text = "The capital of <mask> is Paris."
answer = predict_masked_token(example_text)
print(f"Input: {example_text}")
print(f"Predicted answer: {answer}")

# Example for asking a question
question = "New media as software = Recensão <mask>."
answer = predict_masked_token(question)
print(f"Input: {question}")
print(f"Predicted answer: {answer}")


Input: The capital of <mask> is Paris.
Predicted answer: France
Input: New media as software = Recensão <mask>.
Predicted answer: Software


In [23]:
# Import necessary libraries
import os
from transformers import (
    T5Config, T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
)
from datasets import Dataset

# 1. Create a custom T5 configuration
config = T5Config(
    vocab_size=32128,
    n_positions=512,
    d_model=512,
    d_kv=64,
    d_ff=2048,
    num_layers=6,
    num_decoder_layers=6,
    num_heads=8,
    relative_attention_num_buckets=32,
    dropout_rate=0.1,
    layer_norm_epsilon=1e-6,
    initializer_factor=1.0,
    is_encoder_decoder=True,
    pad_token_id=0,
    eos_token_id=1,
    decoder_start_token_id=0  # Set to the pad_token_id
)


# 2. Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# 3. Initialize the T5 model with the custom configuration
model = T5ForConditionalGeneration(config)

# 4. Load and preprocess data with chunks and in-book train-test splits
train_chunks = []
test_chunks = []

for filename in os.listdir('data'):
    if filename.endswith('.txt'):
        with open(os.path.join('data', filename), 'r', encoding='utf-8') as file:
            content = file.read().strip()
            
            # Tokenize the content into tokens of length 512
            tokenized_content = tokenizer.encode(content, truncation=False, return_tensors="pt")[0]
            
            # Split the tokenized content into chunks of 512 tokens
            for i in range(0, len(tokenized_content), 512):
                chunk = tokenized_content[i:i+512]
                
                # Decode the chunk back to text and store it as a chunk
                chunk_text = tokenizer.decode(chunk, skip_special_tokens=True).strip()
                if chunk_text:  # Ensure the chunk is not empty
                    train_chunks.append(chunk_text)
            
            # Split into training and testing chunks (80/20 split)
            split_index = int(len(train_chunks) * 0.8)
            test_chunks.extend(train_chunks[split_index:])
            train_chunks = train_chunks[:split_index]

# Convert the train and test chunks into Dataset objects
train_dataset = Dataset.from_dict({"text": train_chunks})
test_dataset = Dataset.from_dict({"text": test_chunks})


# Tokenize the datasets
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        padding="longest",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    # Set the input as the label for unsupervised learning
    model_inputs["labels"] = model_inputs["input_ids"].clone()
    return model_inputs

# Apply preprocessing to the training and test data
tokenized_train_data = train_dataset.map(preprocess_function, batched=True, remove_columns=["text"])
tokenized_test_data = test_dataset.map(preprocess_function, batched=True, remove_columns=["text"])

# 5. Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,
    per_device_train_batch_size=1,  # Adjust to fit VRAM
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    eval_steps=500,  # Evaluate every 500 steps
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,  # Enable mixed precision
    remove_unused_columns=False  # Ensure all columns are retained
)

# 6. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data
)

# 7. Train the model
trainer.train()

# 8. Save the trained model and tokenizer
trainer.save_model("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")

Map: 100%|██████████| 17708/17708 [00:23<00:00, 755.21 examples/s]
c:\Users\Filip\anaconda3\envs\ASP\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  1%|          | 100/12100 [00:30<1:00:00,  3.33it/s]

{'loss': 6.6963, 'grad_norm': 8.378801345825195, 'learning_rate': 4.958677685950414e-05, 'epoch': 0.82}


  2%|▏         | 200/12100 [01:00<56:47,  3.49it/s]  

{'loss': 5.8958, 'grad_norm': 12.086433410644531, 'learning_rate': 4.917355371900827e-05, 'epoch': 1.65}


  2%|▏         | 300/12100 [01:30<57:43,  3.41it/s]  

...

100%|██████████| 12100/12100 [2:46:15<00:00,  3.61it/s]  

{'loss': 2.2557, 'grad_norm': 14.04565715789795, 'learning_rate': 1.2396694214876035e-08, 'epoch': 99.79}


100%|██████████| 12100/12100 [2:46:16<00:00,  1.21it/s]


{'train_runtime': 9976.7676, 'train_samples_per_second': 4.861, 'train_steps_per_second': 1.213, 'train_loss': 3.1137027835057784, 'epoch': 99.79}


('./fine_tuned_t5\\tokenizer_config.json',
 './fine_tuned_t5\\special_tokens_map.json',
 './fine_tuned_t5\\spiece.model',
 './fine_tuned_t5\\added_tokens.json')

In [18]:
train_chunks

['180 Manuel Portela Software Takes Command accurately revisi ts and summarizes the history of human -computer interaction, while analyzing these interactions as they are embodied in our current software. The work develops its own critical language and provides a model for analyzing computer applications, theorizing t hem as formal components that are determinant in the creation of media and in the transcoding of cultural practices that incorporate media software. The intelligence and breadth of Manovich’s approach makes this book relevant for all those who, in the fiel ds of computer science, art, design, history and theory of new media, and related disciplines, wish to understand the multiple forms and structures of interaction and manipulation encoded in the software we use in many of our creative and communicative pra ctices. Manovich beautifully synthesizes a significant part of his work as programmer, designer and digital animator, media artist, researcher and professor, helping 

In [25]:
# Import necessary libraries
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 1. Load the trained model and tokenizer
model_path = "./fine_tuned_t5"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# 2. Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Prepare the input text for inference
input_text = "Jaka strategia ta kierowała Warholem już w wywiadzie z 1963 roku?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# 4. Generate predictions with appropriate generation parameters
try:
    outputs = model.generate(
        input_ids,
        max_length=512,       # Maximum length of the generated text
        num_return_sequences=1,
        temperature=0.9,      # Adjusts randomness (increase for more diverse output)
        top_k=50,             # Top-k sampling to limit possible tokens
        top_p=0.95,           # Nucleus sampling for balanced generation
        num_beams=5,          # Use beam search for coherent results
        early_stopping=True,
        no_repeat_ngram_size=2  # Prevents repeating n-grams for more coherent output
    )

    # 5. Decode and print the generated output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Generated Output:", generated_text)

except Exception as e:
    print("Error during generation:", str(e))


Generated Output: a w roku z gldaszt ia. W k o naakiej si cym zasy, którzym dody czytelazu Muzeumaem. Krzto tywyka wana dobdzie najwaanianie ej rozbrze – uzowa byaowanizuca sazkadzia latwartuki. Jegosza ju kiednystawórzuje si
